# Demo Complex Process Synchronization
In the Process Synchronization notebook the Synchronization using **start_events** was compared with a sequence of activities. However, the synchronization of two processes e.g. two vessels can not be implemented using sequence, but requires the **start_event** synchronization. 

In this demo two processes each consisting of a sequence of activities representing the processes of two vessels are synchronized, e.g. because they have to make a hand-over of material. 

This basic mechanism is also used in the feeder demo.

In [28]:
import datetime, time
import simpy

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

## Definition of two sequences of basic activities
The activity names of the first sequence all start with *A Basic activity*. Activity *A Basic activity3* is constrained such that it can only be executed after *B Basic activity 2* has been completed. Thus, in this demo, two processes controlled by a structural control element are synchronized with each other using **start_events**.

In [29]:
reporting_activity_data = {
    "env": my_env,
    "name": "Reporting activity",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5k",  # For logging purposes
    "registry": registry,
    "duration": 0,
    "postpone_start": False,
    "keep_resources": keep_resources,
}
reporting_activity = model.BasicActivity(**reporting_activity_data)

sub_processes = []
basic_activity_data1 = {
    "env": my_env,
    "name": "A Basic activity1",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5b",  # For logging purposes
    "registry": registry,
    "duration": 14,
    "postpone_start": True,
    "additional_logs": [reporting_activity],
    "keep_resources": keep_resources,
}
sub_processes.append(model.BasicActivity(**basic_activity_data1))
basic_activity_data2 = {
    "env": my_env,
    "name": "A Basic activity2",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5c",  # For logging purposes
    "registry": registry,
    "duration": 10,
    "additional_logs": [reporting_activity],
    "postpone_start": True,
    "keep_resources": keep_resources,
}
sub_processes.append(model.BasicActivity(**basic_activity_data2))
basic_activity_data3 = {
    "env": my_env,
    "name": "A Basic activity3",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5d",  # For logging purposes
    "registry": registry,
    "duration": 220,
    "additional_logs": [reporting_activity],
    "postpone_start": True,
    "keep_resources": keep_resources,
    "start_event": [{"type":"activity", "name": "B Basic activity2" , "state":"done"}],
}
sub_processes.append(model.BasicActivity(**basic_activity_data3))

sequential_activity_data = {
    "env": my_env,
    "name": "A Sequential process",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",  # For logging purposes
    "registry": registry,
    "sub_processes": sub_processes,
    "keep_resources": keep_resources,
}
activity = model.SequentialActivity(**sequential_activity_data)

start event instance None
start event instance None


Process 2 definition

In [30]:
sub_processes2 = []
basic_activity_data4 = {
    "env": my_env,
    "name": "B Basic activity1",
    "ID": "5dbbbdf7-4589-11e9-bf3b-b469212bff5b",  # For logging purposes
    "registry": registry,
    "duration": 1,
    "postpone_start": True,
    "additional_logs": [reporting_activity],
    "keep_resources": keep_resources,
}
sub_processes2.append(model.BasicActivity(**basic_activity_data4))
basic_activity_data5 = {
    "env": my_env,
    "name": "B Basic activity2",
    "ID": "5dbbbdf7-4589-11e9-bf3b-b469212bff5c",  # For logging purposes
    "registry": registry,
    "duration": 500,
    "additional_logs": [reporting_activity],
    "postpone_start": True,
    "keep_resources": keep_resources,
}
sub_processes2.append(model.BasicActivity(**basic_activity_data5))
basic_activity_data6 = {
    "env": my_env,
    "name": "B Basic activity3",
    "ID": "5dbbbdf7-4589-11e9-bf3b-b469212bff5d",  # For logging purposes
    "registry": registry,
    "duration": 120,
    "additional_logs": [reporting_activity],
    "postpone_start": True,
    "keep_resources": keep_resources,
}
sub_processes2.append(model.BasicActivity(**basic_activity_data6))

sequential_activity_data2 = {
    "env": my_env,
    "name": "B Sequential process",
    "ID": "5dbbbdf7-4589-11e9-bf3b-b469212bff60",  # For logging purposes
    "registry": registry,
    "sub_processes": (proc for proc in sub_processes2),
    "keep_resources": keep_resources,
}
activity2 = model.SequentialActivity(**sequential_activity_data2)

start event instance None


In [31]:
my_env.run()


keep_resources {}
start event instance None
keep_resources {}
start event instance None
Activity end()
keep_resources {}
start event instance None
Activity end()
keep_resources {}
start event instance None
Activity end()
keep_resources {}
start event expression [{'type': 'activity', 'name': 'B Basic activity2', 'state': 'done'}]
start event <Event() object at 0x24b01c26d48>
start event instance <Event() object at 0x24b01c26d48>
delayed process : <Event() object at 0x24b01c26d48>
Activity end()
delayed process : after yield True
delayed process start subprocess functools.partial(<function basic_process at 0x0000024B7C9BB9D8>, name='A Basic activity3', duration=220, additional_logs=[<openclsim.model.BasicActivity object at 0x0000024B01C606C8>], requested_resources=[], keep_resources={})
keep_resources {}
start event instance None
Activity end()
Activity end()


In [32]:
log_df = pd.DataFrame(reporting_activity.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data_rep = data.drop_duplicates()
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,Reporting activity,START,1970-01-01 01:00:00,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5k
1,A Basic activity1,START,1970-01-01 01:00:00,14,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
2,B Basic activity1,START,1970-01-01 01:00:00,1,5dbbbdf7-4589-11e9-bf3b-b469212bff5b
3,Reporting activity,STOP,1970-01-01 01:00:00,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5k
4,B Basic activity1,STOP,1970-01-01 01:00:01,1,5dbbbdf7-4589-11e9-bf3b-b469212bff5b
5,B Basic activity2,START,1970-01-01 01:00:01,500,5dbbbdf7-4589-11e9-bf3b-b469212bff5c
6,A Basic activity1,STOP,1970-01-01 01:00:14,14,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
7,A Basic activity2,START,1970-01-01 01:00:14,10,6dbbbdf7-4589-11e9-bf3b-b469212bff5c
8,A Basic activity2,STOP,1970-01-01 01:00:24,10,6dbbbdf7-4589-11e9-bf3b-b469212bff5c
9,A Basic activity3,WAIT_START,1970-01-01 01:00:24,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5d


Both processes start at the same time. *A Basic activity3* enters waiting state since *A Basic activity2* has not been completed yet.

**Hint:** Having a start_event based on the last activity of a sequence is currently not working. If the constraint is made based on the end of the parent sequence, then the intended synchronization can be accomplished.

When changing the duration of *A Basic activity2* to 1000 then there will be no waiting since the *A Basic activity2* completes after *B Basic activity2* has been completed. 